In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge

In [5]:
col_types = {'Store': np.int8,
             'Date':str,
             'Weekly_Sales':np.float64,
             'Holiday_Flag':np.float16,
             'Temperature':np.float16,
             'Fuel_Price':np.float16,
             'CPI':np.float16,
            'Unemployment':np.float16,}
data = pd.read_csv('./../data/raw/Walmart_Store_sales.csv', date_parser=True, usecols=col_types.keys(), dtype=col_types)#,  usecols=col_types.keys(), dtype=col_types)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    int8   
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float16
 4   Temperature   132 non-null    float16
 5   Fuel_Price    136 non-null    float16
 6   CPI           138 non-null    float16
 7   Unemployment  135 non-null    float16
dtypes: float16(5), float64(1), int8(1), object(1)
memory usage: 4.1+ KB


In [7]:
data.describe()

Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
count  150.000000  1.360000e+02    138.000000   132.000000  136.000000   
mean     9.866667  1.249536e+06      0.079712    61.406250    3.322266   
std      6.231191  6.474630e+05      0.271973    18.375000    0.478271   
min      1.000000  2.689290e+05      0.000000    18.796875    2.513672   
25%      4.000000  6.050757e+05      0.000000    45.593750    2.851562   
50%      9.000000  1.261424e+06      0.000000    63.000000    3.451172   
75%     15.750000  1.806386e+06      0.000000    76.312500    3.707031   
max     20.000000  2.771397e+06      1.000000    91.625000    4.191406   

             CPI  Unemployment  
count  138.00000    135.000000  
mean   180.00000      7.601562  
std     40.28125      1.577148  
min    126.12500      5.144531  
25%    132.00000      6.597656  
50%    198.00000      7.468750  
75%    214.87500      8.148438  
max    227.00000     14.312500

### Missing values

In [8]:
def missing_values(data):
    return pd.concat([data.isna().sum(), 
        np.round(data.isna().sum()*100/data.shape[0])], 
        axis=1).rename({0:'count_missing', 1:'%_missing'}, axis=1)

In [9]:
missing_values(data)

count_missing  %_missing
Store                     0        0.0
Date                     18       12.0
Weekly_Sales             14        9.0
Holiday_Flag             12        8.0
Temperature              18       12.0
Fuel_Price               14        9.0
CPI                      12        8.0
Unemployment             15       10.0

### Change Store type to str

In [23]:
data = data.astype({'Store':'str'})

### Split in train and test set

In [24]:
# Drop empty dates
data = data.dropna(subset=['Weekly_Sales', 'Date'], axis=0)
X= data.drop('Weekly_Sales', axis=1)
y= data.loc[:,'Weekly_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [25]:
print(f'Train data')
X_train.shape

Train data


(94, 7)

In [26]:
print(f'Train data')
X_test.shape

Train data


(24, 7)

### Create new features from the date column

In [27]:
from sklearn.base import BaseEstimator, TransformerMixin

#Create a Transformer to use it in the pipeline
class DateAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, date_ix = 1):
        self.date_ix = date_ix
        self.new_features = []
    def fit(self, X, y=None):
        return self #Nothing else to do

    def transform(self, X):
        '''
        X is an array-like of shape (n_samples, 1).  1D array
        '''
        if isinstance(X, pd.DataFrame):
            dates = pd.to_datetime(X.iloc[:, 0])
        else:
            dates = pd.to_datetime(X)
        #dates = pd.to_datetime(X) # old code --> dates = pd.to_datetime(X[self.date_ix])
        year=dates.dt.year #Because of null values, year could be NA
        month=dates.dt.month.astype(np.int8)
        day=dates.dt.day.astype(np.int8)
        dayofweek=dates.dt.dayofweek.astype(np.int8)
        weekday=dates.dt.strftime("%A")
        week=dates.dt.isocalendar().week.astype(np.int8)
        self.new_features=['year', 'month', 'day', 'dayofweek', 'week']
        return np.c_[year.values, month.values, day.values, dayofweek.values, week.values]



In [28]:
attr_adder = DateAttributesAdder(date_ix = 0)
attr_adder.transform(pd.DataFrame(X_train['Date']))


array([[2012,    6,    7,    3,   23],
       [2012,   10,   19,    4,   42],
       [2012,   12,   10,    0,   50],
       [2011,   12,    8,    3,   49],
       [2011,    5,   27,    4,   21],
       [2012,    6,    1,    4,   22],
       [2011,   12,    8,    3,   49],
       [2010,   12,    2,    3,   48],
       [2012,   10,    2,    1,   40],
       [2011,   11,   18,    4,   46],
       [2010,   12,   11,    5,   49],
       [2011,    5,   13,    4,   19],
       [2011,    3,   25,    4,   12],
       [2010,    6,    8,    1,   23],
       [2010,    8,   27,    4,   34],
       [2010,    4,   16,    4,   15],
       [2011,    4,   22,    4,   16],
       [2011,    7,   22,    4,   29],
       [2010,   10,   12,    1,   41],
       [2010,    2,   26,    4,    8],
       [2010,    6,   25,    4,   25],
       [2010,    7,   30,    4,   30],
       [2012,    2,   24,    4,    8],
       [2010,   12,    3,    4,   48],
       [2010,    2,   19,    4,    7],
       [2011,    7,   10,

In [29]:
def explode_date(df:pd.DataFrame):
    data = df.copy()
    if data.dtypes['Date'] == 'object':
        data['Date'] = pd.to_datetime(data['Date'])
    data['year']=data['Date'].dt.year #Because of null values, year could be 
    data['month']=data['Date'].dt.month.astype(np.int8)
    data['day']=data['Date'].dt.day.astype(np.int8)
    data['dayOfWeek']=data['Date'].dt.dayofweek.astype(np.int8)
    data['weekday']=data['Date'].dt.strftime("%A")
    data['week']=data['Date'].dt.isocalendar().week.astype(np.int8)
    return data

In [30]:
#X_train = explode_date(X_train)
#X_train.head()

### Transform holiday flat to int

In [31]:
def float_to_int(x):
    if not np.isnan(x):
        return int(x)
    return np.NaN

### Pre/processing

In [32]:
num_idx = [3, 4, 5, 6 ] #[3, 4, 5, 6, 7, 8, 9, 10, 12]
cat_idx = [0, 2]
date_idx = [1]
num_features = X_train.columns[num_idx] #['Temperature', 'Fuel_Price', 'CPI', Unemployment', 'year', 'month','day', 'dayOfWeek', 'week']
cat_features = X_train.columns[cat_idx] #['Store', 'Holiday_Flag']
date_feature = X_train.columns[date_idx]


In [33]:
# Transformer for extra features from Date
date_transformer = Pipeline([
        #('imputer_date', SimpleImputer(strategy = 'most_frequent')),
        ('attr_adder', DateAttributesAdder(date_ix=0)),
        ('scaler', StandardScaler())
    ])

# transformer for numerical features
num_transformer = Pipeline([
        ('imputer_num', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler())
    ])
# transformer for categorical features
cat_transformer = Pipeline([
        ('imputer_cat', SimpleImputer(strategy = 'most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer([
        ('date', date_transformer, date_feature),
        ('categoricals', cat_transformer, cat_features),
        ('numericals', num_transformer, num_features)
    ],
    remainder = 'drop'
)

In [34]:
#trans_X_train = preprocessor.transformers[0][1][1].get_feature_names(['Store', 'Holiday_Flag'])

In [35]:
full_pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('lin_reg', LinearRegression())
    ])

In [36]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('date',
                                                  Pipeline(steps=[('attr_adder',
                                                                   DateAttributesAdder(date_ix=0)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Date'], dtype='object')),
                                                 ('categoricals',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  

In [37]:
y_pred = full_pipeline.predict(X_train)

In [38]:
from sklearn.metrics import r2_score

In [39]:
print(r2_score(y_train, y_pred))

0.9698245315928727


In [40]:
#X_test = explode_date(X_test)
#X_test_tr = preprocessor.transform(X_test)
y_pred_test = full_pipeline.predict(X_test)
print(r2_score(y_test, y_pred_test))

0.9395265229021438


In [43]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

In [52]:
regressors = [
    KNeighborsRegressor(3),
    #SVC(kernel="rbf", C=0.025, probability=True),
    #DecisionTreeClassifier(),
    RandomForestRegressor(),
    AdaBoostRegressor(),
    Ridge(),
    Lasso(),
   # GradientBoostingClassifier()
    ]
for regressor in regressors:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', regressor)])
    pipe.fit(X_train, y_train)   
    print(regressor)
    print("model score train: %.3f" % pipe.score(X_test, y_test))
    print("model score test: %.3f" % pipe.score(X_test, y_test))

KNeighborsRegressor(n_neighbors=3)
model score train: 0.323
model score test: 0.323
RandomForestRegressor()
model score train: 0.576
model score test: 0.576
AdaBoostRegressor()
model score train: 0.583
model score test: 0.583
Ridge()
model score train: 0.881
model score test: 0.881
Lasso()
model score train: 0.941
model score test: 0.941


/home/carmina/miniconda3/envs/pez-jedha/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 585678915475.783, tolerance: 3893666853.6647453
  positive)
